In [ ]:
import os
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, END
from langchain_community.utilities import GoogleSerperAPIWrapper
from transformers import pipeline
from PIL import Image
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Ключ (мои не брать, уберу)
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "..."
os.environ["SERPER_API_KEY"] = "..."

# Состояние графа
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], "Сообщения"]
    image_path: str
    car_model: str
    history_info: str
    sale_link: str

# Узел 1: Распознавание (марка авто)
def recognize_car(state: AgentState) -> AgentState:
    print("─── Распознавание автомобиля ───")
    print(f"Изображение: {state['image_path']}")
    
    try:
        image = Image.open(state['image_path'])
        classifier = pipeline("image-classification", model="dima806/car_models_image_detection")
        results = classifier(image)
        top_result = results[0]['label'].strip()
        
        # Чистим, если есть лишнее (иногда "class: Toyota")
        if ':' in top_result:
            top_result = top_result.split(':', 1)[1].strip()
        
        state['car_model'] = top_result
        print(f"Марка: {top_result}")
    except Exception as e:
        state['car_model'] = f"Ошибка распознавания: {str(e)}"
        print(state['car_model'])
    
    return state

# Узел 2: Историческая справка 
def get_history(state: AgentState) -> AgentState:
    print("─── Промежуточное состояние: Историческая справка ───")
    print(f"Марка/модель: {state['car_model']}")
    
    try:
        endpoint = HuggingFaceEndpoint(
            repo_id="Qwen/Qwen2.5-7B-Instruct",
            max_new_tokens=300,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1, 
            huggingfacehub_api_token=os.environ.get("HUGGINGFACEHUB_API_TOKEN"),
        )
        
        llm = ChatHuggingFace(llm=endpoint)
        
        # Промпт в чат-формате (system + human)
        prompt = ChatPromptTemplate.from_messages([
            ("system", "Ты — эксперт по истории автомобилей. Пиши только факты, кратко, на английском языке."),
            ("human", "Напиши краткую историческую справку (2–4 предложения) о марке или модели автомобиля на английском языке: {model}"),
        ])
        
        chain = prompt | llm | StrOutputParser()
        
        history = chain.invoke({"model": state['car_model']}).strip()
        
        # Небольшая чистка на всякий случай
        history = history.replace("assistant:", "").strip()
        
        if not history or len(history) < 20:
            history = f"Информация о {state['car_model']} временно недоступна."
        
        state['history_info'] = history
        print(f"Справка:\n{history}")
    
    except Exception as e:
        error_msg = f"Ошибка LLM: {str(e)}"
        state['history_info'] = error_msg
        print(error_msg)
    
    return state

# Узел 3: Поиск объявления
def get_sale_link(state: AgentState) -> AgentState:
    print("─── Поиск объявления ───")
    print(f"Запрос: {state['car_model']}")
    
    try:
        search = GoogleSerperAPIWrapper()
        query = f"{state['car_model']} купить объявление Россия"
        results = search.results(query)
        if results.get('organic') and results['organic']:
            sale_link = results['organic'][0]['link']
        else:
            sale_link = "Объявления не найдены"
    except Exception as e:
        sale_link = f"Ошибка поиска: {str(e)}"
    
    state['sale_link'] = sale_link
    print(f"Ссылка: {sale_link}")
    return state

# Граф
workflow = StateGraph(AgentState)
workflow.add_node("recognize_car", recognize_car)
workflow.add_node("get_history", get_history)
workflow.add_node("get_sale_link", get_sale_link)

workflow.add_edge("recognize_car", "get_history")
workflow.add_edge("get_history", "get_sale_link")
workflow.add_edge("get_sale_link", END)

workflow.set_entry_point("recognize_car")
app = workflow.compile()

# Запуск на папке
def run_car_analysis(folder_path="data_cars"):
    if not os.path.exists(folder_path):
        print(f"Папка {folder_path} не найдена!")
        return
    
    images = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if not images:
        print("Нет изображений в папке data_cars")
        return
    
    for filename in images:
        image_path = os.path.join(folder_path, filename)
        print(f"\n===== Обработка: {filename} =====")
        
        initial_state = {"image_path": image_path}
        try:
            final_state = app.invoke(initial_state)
            print("\nРезультат:")
            print(f"Марка/модель: {final_state.get('car_model', '—')}")
            print(f"Справка:\n{final_state.get('history_info', '—')}")
            print(f"Объявление: {final_state.get('sale_link', '—')}")
        except Exception as e:
            print(f"Общая ошибка: {str(e)}")

if __name__ == "__main__":
    print("Старт анализа...")
    run_car_analysis()